<a href="https://colab.research.google.com/github/pablobots/private_test/blob/main/Pediatric_Bone_Age_EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/pablobots/private_test/blob/main/Pediatric_Bone_Age.ipynb


In [1]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


In [2]:
!pip install -q kaggle

In [3]:
!mkdir ~/.kaggle 

In [4]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [5]:
from google.colab import drive
drive.flush_and_unmount()

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
#!kaggle datasets list

In [8]:
!kaggle datasets download -d kmader/rsna-bone-age -p ./

100% 9.27G/9.29G [03:38<00:00, 43.6MB/s]
100% 9.29G/9.29G [03:38<00:00, 45.6MB/s]


In [9]:
!unzip -qq rsna-bone-age.zip

In [11]:
!rm rsna-bone-age.zip
!rm -r boneage-test-dataset
!rm boneage-test-dataset.csv

In [10]:
import os
from PIL import Image

size=224
input_dir = "/content/boneage-training-dataset/boneage-training-dataset"
dirs = os.listdir(input_dir)
for item in dirs:
  image_path = os.path.join(input_dir, item)
  if os.path.isfile(image_path) and image_path.endswith('.png'):
    im = Image.open(image_path)
    imResize = im.resize((size,size), Image.ANTIALIAS)
    imResize.save(image_path, 'PNG', quality=90)
    #print(image_path)

In [12]:
!rm -r boneage-dataset-deit
!mkdir boneage-dataset-deit
!mkdir boneage-dataset-deit/train
!mkdir boneage-dataset-deit/val

rm: cannot remove 'boneage-dataset-deit': No such file or directory


In [13]:
import pandas as pd
import shutil
import random

input_dir = "/content/boneage-training-dataset/boneage-training-dataset"
output_dir_train = "boneage-dataset-deit/train"
output_dir_val = "boneage-dataset-deit/val"
df = pd.read_csv('boneage-training-dataset.csv')

for index, row in df.iterrows():
  #print(row['id'], row['boneage'], row['male'])
  
  file_path = os.path.join(input_dir, str(row['id']) + ".png")
  boneage = str(row['boneage'])

  output_subdir_train = os.path.join(output_dir_train, boneage)
  if not os.path.exists(output_subdir_train):
    os.mkdir(output_subdir_train)
  output_subdir_val = os.path.join(output_dir_val, boneage)
  if not os.path.exists(output_subdir_val):
    os.mkdir(output_subdir_val)

  if random.uniform(0, 1) <= 0.85:
    shutil.copy(file_path, output_subdir_train)
  else:
    shutil.copy(file_path, output_subdir_val)

In [14]:
!git clone https://github.com/facebookresearch/deit.git

Cloning into 'deit'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 101 (delta 3), reused 9 (delta 2), pack-reused 89
Receiving objects: 100% (101/101), 204.55 KiB | 22.73 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [15]:
%cd deit/

/content/deit


In [16]:
!pip install torch torchvision

In [17]:
!pip install timm==0.3.2

     |████████████████████████████████| 245kB 17.2MB/s 


In [22]:
!python main.py --model deit_small_patch16_224 --batch-size 64 --epochs 100 --output_dir /content/deit/output --data-path /content/boneage-dataset-deit

Not using distributed mode
Namespace(aa='rand-m9-mstd0.5-inc1', batch_size=64, clip_grad=None, color_jitter=0.4, cooldown_epochs=10, cutmix=1.0, cutmix_minmax=None, data_path='/content/boneage-dataset-deit', data_set='IMNET', decay_epochs=30, decay_rate=0.1, device='cuda', dist_eval=False, dist_url='env://', distillation_alpha=0.5, distillation_tau=1.0, distillation_type='none', distributed=False, drop=0.0, drop_path=0.1, epochs=100, eval=False, inat_category='name', input_size=224, lr=0.0005, lr_noise=None, lr_noise_pct=0.67, lr_noise_std=1.0, min_lr=1e-05, mixup=0.8, mixup_mode='batch', mixup_prob=1.0, mixup_switch_prob=0.5, model='deit_small_patch16_224', model_ema=True, model_ema_decay=0.99996, model_ema_force_cpu=False, momentum=0.9, num_workers=10, opt='adamw', opt_betas=None, opt_eps=1e-08, output_dir='/content/deit/output', patience_epochs=10, pin_mem=True, recount=1, remode='pixel', repeated_aug=True, reprob=0.25, resplit=False, resume='', sched='cosine', seed=0, smoothing=0.1

In [ ]:
!python main.py --model deit_small_patch16_224 --batch-size 64 --epochs 200 --resume /content/deit/output/checkpoint.pth --output_dir /content/deit/output --data-path /content/boneage-dataset-deit --mixup 0 --cutmix 0 --mixup-prob 0 --mixup-switch-prob 0 --smoothing 0

Not using distributed mode
Namespace(aa='rand-m9-mstd0.5-inc1', batch_size=64, clip_grad=None, color_jitter=0.4, cooldown_epochs=10, cutmix=0.0, cutmix_minmax=None, data_path='/content/boneage-dataset-deit', data_set='IMNET', decay_epochs=30, decay_rate=0.1, device='cuda', dist_eval=False, dist_url='env://', distillation_alpha=0.5, distillation_tau=1.0, distillation_type='none', distributed=False, drop=0.0, drop_path=0.1, epochs=200, eval=False, inat_category='name', input_size=224, lr=0.0005, lr_noise=None, lr_noise_pct=0.67, lr_noise_std=1.0, min_lr=1e-05, mixup=0.0, mixup_mode='batch', mixup_prob=0.0, mixup_switch_prob=0.0, model='deit_small_patch16_224', model_ema=True, model_ema_decay=0.99996, model_ema_force_cpu=False, momentum=0.9, num_workers=10, opt='adamw', opt_betas=None, opt_eps=1e-08, output_dir='/content/deit/output', patience_epochs=10, pin_mem=True, recount=1, remode='pixel', repeated_aug=True, reprob=0.25, resplit=False, resume='/content/deit/output/checkpoint.pth', sc

In [ ]:
!python main.py --eval --resume /content/deit/output/checkpoint.pth --model deit_small_patch16_224 --batch-size 32 --data-path /content/boneage-dataset-deit

In [21]:
from PIL import Image
import torch
import timm

import torchvision.transforms as transforms

from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

model = torch.hub.load('facebookresearch/deit:main', 'deit_small_patch16_224', pretrained=False)
ckpt_path = '/content/deit/output/checkpoint.pth'

ckpt_dict = torch.load(ckpt_path)
model.load_state_dict(ckpt_dict["model"])

model.eval()

transform = transforms.Compose([
    transforms.Resize(256, interpolation=3),  # to maintain same ratio w.r.t. 224 images
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
])

img_path = '/content/dataset_3/test/capsella_bursa_pastoris/159.png'
#img_path = '/content/dataset_3/test/scentless_mayweed/10.png'
#img_path = '/content/dataset_3/test/small_flowered_cranesbill/1.png'
img = Image.open(img_path).convert("RGB")
img = transform(img)[None,]

out = model(img)
pred_label = torch.argmax(out)
print(pred_label.item())


Downloading: "https://github.com/facebookresearch/deit/archive/main.zip" to /root/.cache/torch/hub/main.zip


FileNotFoundError: ignored